$ min \, f(\vec{x} ) = -20 \cdot e^{\left(-0.2\sqrt{\frac{1}{n} \sum^{n}_{i=1} x^2_1} \right)} -e^{\left( \frac{1}{n} \sum^n_{i=1} cos(2\pi x_i \right)} + 20 + e$     
donde $|x_i| <= 30$

In [3]:
from math import e,cos,pi
import numpy as np
from numpy.random import normal, uniform
from random import sample
from statistics import mean, pstdev
from numpy import ndarray as nd

In [4]:
def f(x):
    n = len(x)
    #try:
    return -20*e**(-0.2*((1/n)*sum(x_i**2 for x_i in x))) - e**((1/n)*sum(cos(2*pi*x_i) for x_i in x)) + 20 + e     
    #except:
    #    print(x)

In [5]:
f([8.63110648, -29.11404831])

21.680282958904787

## (1+1)-EE (20 puntos)

In [6]:
def random_solution(n):
    parent = []
    x_s = []
    for i in range(n):
        x_s.append(uniform(-30,30))
    sigma = uniform()
    return [np.array(x_s), sigma, f(x_s)]

In [33]:
random_solution(2)

[array([-27.92920613, -18.56055323]), 0.029994013889005577, 21.731098608822855]

In [7]:
def check_domain(x,lower_limit, upper_limit):
    x = list(x)
    for i in range(len(x)):
        if x[i] < lower_limit:
            x[i] = lower_limit
        elif x[i] > upper_limit:
            x[i] = upper_limit
    return np.array(x)

In [8]:
def mutate_sol(solution):
    x, sigma, fx = solution
    x = list(x)
    for i in range(len(x)):
        x[i] += sigma*normal(0,1) 
    x = check_domain(x,-30,30)
    return [x,sigma,f(x)]

In [9]:
def EE1plus1(n, c, gens, k, sigma=None):
    if sigma==None: sigma=uniform()
    best_sol = random_solution(n)
    success_count = 0
    for i in range(gens):
        mutated_sol = mutate_sol(best_sol)
        if mutated_sol[2] < best_sol[2]:
            best_sol = mutated_sol
            success_count += 1
        if i%k == 0:
            success_rate = success_count/k
            if success_rate > 1/5:
                sigma /= c
            elif success_rate < 1/5:
                sigma *= c
            success_count = 0
    return best_sol

In [10]:
EE1plus1(2,.8,1000,50)

[array([30.        , -3.00063548]), 0.9145443368758632, 20.00001083421377]

In [11]:
print('Best f =',min([EE1plus1(2,.8,1000,50)[2] for i in range(100)]))

Best f = 0.00017255001043681162


### Statistical analysis

In [14]:
def selection(new_size, population):
    population.sort(key=lambda x: x[2])
    return population[:new_size]

In [15]:
def m_EE1plus1(m, n, c, gens, k, sigma=None):
    m_solutions = []
    for i in range(m):
        EE_sol = EE1plus1(n, c, gens, k, sigma)
        m_solutions.append(EE_sol)
    selection(m, m_solutions)
    return m_solutions

In the function above I've already ordered the solutions from best to worst.

In [23]:
m_EE1plus1(5, 2,.8,200,50)

[[array([-20.99120532, -15.9993924 ]), 0.8111160964064357, 20.00208365683298],
 [array([-19.00098381,  12.01203027]),
  0.40211976546633743,
  20.003904108910284],
 [array([28.98883039, 13.98444014]), 0.5559707619786525, 20.009818186467747],
 [array([27.97261138, 12.99666358]), 0.3100056517588514, 20.020297788831236],
 [array([-16.92216665, -16.97237747]), 0.4606006448104105, 20.173919281483666]]

In [24]:
_20sols = m_EE1plus1(20, 2,.8,200,50)

In [27]:
def median_sol(m,solutions,one_solution=True):
    if m%2 == 0:
        if one_solution:
            return solutions[m//2]
        return solutions[(m//2)-1] + solutions[(m//2)]
    else:
        return solutions[m//2]

In [29]:
median_sol(20,_20sols)

[array([-26.97645594,  23.00912954]), 0.2442840197457442, 20.017026420852904]

In [30]:
def mean_sol_f(solutions):
    return mean([s[2] for s in solutions])

In [31]:
mean_sol_f(_20sols)

19.062267373788107

In [32]:
def std_sol_f(solutions):
    return pstdev([s[2] for s in solutions])

In [33]:
std_sol_f(_20sols)

4.262529192357316

In [35]:
def m_EE1plus1_statistical_analysis(m, n, c, gens, k, sigma=None):
    m_sols =  m_EE1plus1(m, n, c, gens, k, sigma)
    print('Best solution:',m_sols[0])
    print('Worst solution:', m_sols[-1])
    print('Median solution:', median_sol(m,m_sols))
    print('Mean solution:', mean_sol_f(m_sols))
    print('Standard deviation:', std_sol_f(m_sols))

In [42]:
m = 20
m_EE1plus1_statistical_analysis(m, 2, 0.8, 100, 50)

Best solution: [array([0.09832494, 0.0109941 ]), 0.5044061393885332, 0.2624874255553924]
Worst solution: [array([ 17.05660006, -13.94235064]), 0.6338881632896393, 20.167831753523437]
Median solution: [array([28.98619294,  9.01616902]), 0.7265252683320206, 20.012092146832863]
Mean solution: 18.961173126662032
Standard deviation: 4.3059502329108845


## $(\mu, \lambda)- EE$

In [43]:
def rand_select_parents(n, population, n_parents):
    parents_index = sample([i for i in range(n)], n_parents)
    return [population[parents_index[0]], population[parents_index[1]]]

In [44]:
rand_select_parents(9,[21,41,44,4323,37,45,3,44,43],2)

[37, 21]

In [45]:
def discrete_offspring(n, parents):
    x_s = []
    sigma = 0
    for i in range(n):
        if uniform() < 0.5:
            x_s.append(parents[0][0][i])
        else:
            x_s.append(parents[1][0][i])
    if uniform() < 0.5:
        sigma = parents[0][1]
    else:
        sigma = parents[1][1]
    return [np.array(x_s), sigma, 0]#not necessary to calculate f(x) because we still have to mutate after this, we can  calculate it then.    

In [46]:
discrete_offspring(3,[ [ [3,4,5],4,4 ], [ [6,7,8],3,3 ] ])

[array([6, 4, 8]), 4, 0]

Mutation with a unique step size:    
$\sigma' = \sigma \cdot e^{\tau \, \cdot N(0,1)}$      
$x'_i = x_i + \sigma'_i \cdot N(0,1)$      
$ \tau = \frac{1}{\sqrt{n}}$

In [47]:
def mutate(offspring_i, sigma, n, tau=None):
    if tau == None: tau = 1/(n**(1/2)) 
    sigma *= (e**(tau*normal())) 
    x_s = []
    for i in range(n):
        new_x_i = offspring_i[0][i] + sigma * normal()
        x_s.append(new_x_i)
    return [np.array(x_s), sigma, f(x_s)] 

In [49]:
rand_sol = random_solution(2)
mutated_solution = mutate(rand_sol, uniform(), 2)
print(rand_sol)
print(mutated_solution)

[array([-15.85301952, -21.56171254]), 0.651869184002844, 21.867302245577886]
[array([-15.08044475, -21.98884241]), 0.2576068867225153, 20.167881516745556]


In [50]:
def EEmu_coma_lambda(n, gens, epsilon, mu, lambda_,sigma=None, tau=None):
    if sigma==None: sigma=uniform()
    population = [random_solution(n) for i in range(mu)]
    for gen in range(gens):
        offspring = []
        for kid in range(lambda_):
            parents = rand_select_parents(mu, population, 2) # here can I select the same parent twice? I guess because there are more children than parents.
            offspring_i = discrete_offspring(n,parents)
            offspring_i = mutate(offspring_i,sigma,n,tau)
            offspring_i[0] = check_domain(offspring_i[0],-30,30)
            offspring.append(offspring_i)
        population = selection(lambda_, offspring)
    return population[0]

In [51]:
EEmu_coma_lambda(2, 200, 0.0001, 100, 700)

[array([-0.01899486, -0.00279261]), 0.3251112353143156, 0.010596801914183107]

### Statistical analysis

In [52]:
def m_EEmu_coma_lambda(m, n, gens, epsilon, mu, lambda_,sigma=None, tau=None):
    m_solutions = []
    for i in range(m):
        EE_sol = EEmu_coma_lambda(n, gens, epsilon, mu, lambda_,sigma, tau)
        m_solutions.append(EE_sol)
    selection(m, m_solutions)
    return m_solutions

In the function above I've already ordered the solutions from best to worst.

In [53]:
m_EEmu_coma_lambda(5, 2, 200, 0.0001, 100, 700)

[[array([ 0.00141381, -0.00153111]),
  0.010889901419058294,
  0.0001252030322622666],
 [array([-0.0007387 ,  0.00204452]),
  0.06887531642215033,
  0.00013623030587739393],
 [array([-0.00116009,  0.00725814]),
  0.06578579658928135,
  0.001556859742575245],
 [array([-0.01122875,  0.00450985]), 0.8332871025386054, 0.004216874387495739],
 [array([0.00218316, 0.04239795]), 0.16536013171430414, 0.051251494565001554]]

In [54]:
def m_EEmu_coma_lambda_statistical_analysis(m, n, gens, epsilon, mu, lambda_,sigma=None, tau=None):
    m_sols =  m_EEmu_coma_lambda(m, n, gens, epsilon, mu, lambda_, sigma, tau)
    print('Best solution:',m_sols[0])
    print('Worst solution:', m_sols[-1])
    print('Median solution:', median_sol(m,m_sols))
    print('Mean solution:', mean_sol_f(m_sols))
    print('Standard deviation:', std_sol_f(m_sols))

In [55]:
m = 15
m_EEmu_coma_lambda_statistical_analysis(m, 2, 20, 0.0001, 100, 700)

Best solution: [array([ 0.00018846, -0.00237785]), 0.1462163265115881, 0.00016401740614968574]
Worst solution: [array([0.04219404, 0.00212871]), 1.052198184474461, 0.05076098407605345]
Median solution: [array([-0.02714102, -0.01460403]), 0.17763763198254037, 0.02721387431293154]
Mean solution: 0.023484351063723558
Standard deviation: 0.01821648927329441


## $(\mu, \lambda)- EE*$ (*improved) (40 puntos)
\*by using intermediate local recombination

In [56]:
def intermediate_offspring(parents_sols):
    mean_x_s = (parents_sols[0][0]+parents_sols[1][0])/2
    mean_sigma = (parents_sols[0][1]+parents_sols[1][1])/2
    return [mean_x_s,mean_sigma,0] #f(x) calculated in the next step

In [57]:
n = 3
rand_sol = random_solution(n)
mutated_solution = mutate(rand_sol, uniform(), n)
print(rand_sol)
print(mutated_solution)
print(intermediate_offspring([rand_sol, mutated_solution]))

[array([-22.90191377, -26.92328303,   4.86843568]), 0.575863992026013, 20.507946688033368]
[array([-22.37538659, -26.4968742 ,   4.99135638]), 0.9213068357532083, 21.929054804216392]
[array([-22.63865018, -26.71007861,   4.92989603]), 0.7485854138896106, 0]


In [59]:
def EEmu_coma_lambda_2(n, gens, epsilon, mu, lambda_,sigma=None, tau=None):
    if sigma==None: sigma=uniform()
    population = [random_solution(n) for i in range(mu)]
    for gen in range(gens):
        offspring = []
        for kid in range(lambda_):
            parents = rand_select_parents(mu, population, 2) # here can I select the same parent twice? I guess because there are more children than parents.
            offspring_i = intermediate_offspring(parents)
            offspring_i = mutate(offspring_i,sigma,n,tau)
            offspring_i[0] = check_domain(offspring_i[0],-30,30)
            offspring.append(offspring_i)
        population = selection(lambda_, offspring)
    return population[0]

In [60]:
EEmu_coma_lambda_2(2, 200, 0.0001, 100, 700)

[array([ 0.0049861 , -0.00595025]), 0.08979191562974081, 0.0017367451982903681]

## Statistical analysis (20 puntos)

In [61]:
def m_EEmu_coma_lambda_2(m, n, gens, epsilon, mu, lambda_,sigma=None, tau=None):
    m_solutions = []
    for i in range(m):
        EE_sol = EEmu_coma_lambda_2(n, gens, epsilon, mu, lambda_, sigma, tau)
        m_solutions.append(EE_sol)
    selection(m, m_solutions)
    return m_solutions

In the function above I've already ordered the solutions from best to worst.

In [62]:
m_EEmu_coma_lambda_2(5, 2, 200, 0.0001, 100, 700)

[[array([-0.0010895 , -0.00095038]),
  0.10725541688790168,
  6.025707849177664e-05],
 [array([-0.0021912 , -0.00606292]),
  0.23940556926956755,
  0.0011977658337767316],
 [array([-0.01114906, -0.00499251]), 0.2170329083156825, 0.004297597254446561],
 [array([-0.0130974 , -0.00230412]), 0.3347663357549036, 0.0050915875422608],
 [array([0.03199039, 0.02481124]), 0.21683707246340106, 0.04677147286508587]]

In [63]:
_20sols = m_EEmu_coma_lambda_2(20, 2, 200, 0.0001, 100, 700)

In [64]:
median_sol(20,_20sols)

[array([0.01082738, 0.00011594]), 0.17916525360312976, 0.0033769682395683454]

In [65]:
mean_sol_f(_20sols)

0.0072563606613457845

In [66]:
std_sol_f(_20sols)

0.009767613529642097

In [67]:
def m_EEmu_coma_lambda_2_statistical_analysis(m, n, gens, epsilon, mu, lambda_,sigma=None, tau=None):
    m_sols =  m_EEmu_coma_lambda_2(m, n, gens, epsilon, mu, lambda_,sigma, tau)
    print('Best solution:',m_sols[0])
    print('Worst solution:', m_sols[-1])
    print('Median solution:', median_sol(m,m_sols))
    print('Mean solution:', mean_sol_f(m_sols))
    print('Standard deviation:', std_sol_f(m_sols))

In [68]:
m = 20
m_EEmu_coma_lambda_2_statistical_analysis(m, 2, 200, 0.0001, 100, 700)

Best solution: [array([-0.00023696, -0.00043037]), 0.004911791952664991, 6.958290385217225e-06]
Worst solution: [array([ 0.02790946, -0.02995332]), 0.21283747416990165, 0.04782774113542887]
Median solution: [array([-0.00771624,  0.00782658]), 0.22723142093863455, 0.0034797692219865617]
Mean solution: 0.012342289979257348
Standard deviation: 0.015506322764773723


In [69]:
m = 20
m_EEmu_coma_lambda_2_statistical_analysis(m, 2, 20, 0.0001, 100, 700)

Best solution: [array([-2.09009880e-06, -1.96037773e-05]), 0.00023815529404117036, 1.1204914773799146e-08]
Worst solution: [array([0.04424599, 0.05696006]), 0.6613812763372181, 0.14524499155293968]
Median solution: [array([-0.00060332,  0.0130858 ]), 0.17815661872949135, 0.004940525778363014]
Mean solution: 0.020750507688578158
Standard deviation: 0.03881912748224787


### Examples varying only number of decision variables 

#### Example with 5 decision variables

In [70]:
m = 10
m_EEmu_coma_lambda_2_statistical_analysis(m, 5, 20, 0.0001, 100, 700)

Best solution: [array([-0.01875889, -0.00209488,  0.0063667 , -0.01650577, -0.03782835]), 0.07574582863476517, 0.02404683760228865]
Worst solution: [array([ 0.05907556,  0.0792718 , -0.13065419, -0.32690491, -0.06127563]), 0.5543332880704767, 1.022477391692171]
Median solution: [array([ 0.11049454, -0.02414419,  0.01896506, -0.08506619,  0.17164696]), 0.43273282275377206, 0.49526462079136424]
Mean solution: 0.43853672354403245
Standard deviation: 0.38500556733991725


#### Example with 10 decision variables

In [71]:
m = 10
m_EEmu_coma_lambda_2_statistical_analysis(m, 10, 20, 0.0001, 100, 700)

Best solution: [array([-0.06633252,  0.01547283,  0.04455212,  0.02092215,  0.01045776,
       -0.02292581, -0.00885292, -0.0865039 , -0.03483651, -0.00503343]), 0.07793799404178317, 0.09231250022664073]
Worst solution: [array([-0.44379848,  0.12081892,  0.01959369, -0.4805241 ,  0.06038358,
       -0.88856705, -0.91329219,  0.06919511, -0.04008266, -0.05173417]), 0.7146951677248259, 1.857772095642336]
Median solution: [array([-0.03594864,  0.28646686, -0.03098353, -0.18270489, -0.01878661,
        0.07298074,  0.00253748,  0.69048225,  0.08620184,  0.06310283]), 0.40464075964348806, 1.0568538074341949]
Mean solution: 0.9458855462650899
Standard deviation: 0.604401720973657


#### Example with 20 dicision variables

In [72]:
m = 10
m_EEmu_coma_lambda_2_statistical_analysis(m, 20, 20, 0.0001, 100, 700)

Best solution: [array([-0.01606896,  0.8598667 , -0.05874008, -0.02558589,  0.05124576,
        0.9090163 ,  0.07928926, -0.89400049,  0.00975405,  0.02847225,
        0.05350045,  0.85999989, -0.05943743,  1.00267508,  0.01743497,
       -0.02596027, -0.05545758,  0.0443054 ,  0.89538858, -0.13542247]), 0.07180248517859412, 1.2462150176981308]
Worst solution: [array([-0.47309579, -0.10748281,  0.21150759, -0.46995158, -0.44698657,
        0.55760978, -0.70705786, -0.02196189, -0.1161981 , -0.65405531,
       -0.71254951, -0.06155033,  0.25769837, -0.71962613,  0.47481447,
        1.10342142, -0.65879766, -0.32795678,  0.25681481,  0.82824776]), 0.4948466190106547, 2.9487985424537944]
Median solution: [array([-0.60747024,  0.4554271 , -0.75850122,  0.38845421,  0.64704422,
       -0.15085822,  1.00864432, -0.62581901, -0.34002263, -0.07318052,
       -0.00222501, -0.0361538 , -0.26621091,  0.00785994, -0.21520884,
        0.14695885,  0.09180943,  0.53491257, -0.17958924, -0.26041013])

## Comparison:  (1+1)-EE      vs  $(\mu, \lambda)- EE$  vs  $(\mu, \lambda)- EE*$ (20 puntos)

In [73]:
m = 20
gens = 200
epsilon = 0.0001
k = 50
c = 0.8
mu = 100
lambda_ = 700

### 2 variables

In [91]:
n = 2

In [75]:
m_EE1plus1_statistical_analysis(m, n, c, gens, k)

Best solution: [array([0.0865936 , 0.12891703]), 0.7397472262400376, 0.6011862010882258]
Worst solution: [array([21.05497376, 10.02392313]), 0.7061659012891763, 20.09393931992774]
Median solution: [array([ 23.01539441, -15.01616318]), 0.33742382350211664, 20.013323134260368]
Mean solution: 19.051440173046764
Standard deviation: 4.232848246692949


In [76]:
m_EEmu_coma_lambda_statistical_analysis(m, n, gens, epsilon, mu, lambda_)

Best solution: [array([-0.00107328,  0.00026563]), 0.015650995826076007, 3.524187670267409e-05]
Worst solution: [array([-0.01654252,  0.07612878]), 0.15173847858035067, 0.16740314303378812]
Median solution: [array([-0.01111524, -0.0104042 ]), 0.14118163207094178, 0.006672806889398419]
Mean solution: 0.02358493990593393
Standard deviation: 0.03927134480025898


In [92]:
m_EEmu_coma_lambda_2_statistical_analysis(m, n, gens, epsilon, mu, lambda_)

Best solution: [array([ 0.00037108, -0.00042753]), 0.02369128786666519, 9.238938308175904e-06]
Worst solution: [array([0.01903055, 0.03421533]), 0.1293105839946387, 0.04374946843978167]
Median solution: [array([ 0.01630398, -0.00215567]), 0.13297874154338746, 0.00778121289218392]
Mean solution: 0.009995132216457935
Standard deviation: 0.0117439134574944


### 5 variables

In [78]:
n = 5

In [80]:
m_EE1plus1_statistical_analysis(m, n, c, gens, k)

Best solution: [array([ -2.00726387,  -9.99488076, -19.99574813,  25.00083966,
       -14.99765244]), 0.010106218260058863, 20.001107809859562]
Worst solution: [array([  1.99830825, -13.1120682 , -15.79587381,   5.91510282,
        27.15290365]), 0.9062240891061878, 20.712420824131915]
Median solution: [array([29.03205573, -8.97083096,  0.97348608, 28.14096976, 14.14890961]), 0.28780091859114176, 20.4133643891098]
Mean solution: 20.384016904738335
Standard deviation: 0.20615871338139483


In [81]:
m_EEmu_coma_lambda_statistical_analysis(m, n, gens, epsilon, mu, lambda_)

Best solution: [array([ 0.0114716 , -0.01098193,  0.01322591, -0.01336231, -0.01921267]), 0.10163567388559804, 0.01121256440277607]
Worst solution: [array([ 15.09106577,  19.94032199, -21.95146468, -30.        ,
        16.21942336]), 0.6771581306766467, 20.53798145638035]
Median solution: [array([-0.01449358, -0.03801741,  0.72026126,  0.03610016,  0.07697692]), 0.45676901896641053, 1.0644810806416385]
Mean solution: 6.472555886382096
Standard deviation: 8.982425850374126


In [82]:
m_EEmu_coma_lambda_2_statistical_analysis(m, n, gens, epsilon, mu, lambda_)

Best solution: [array([-4.35157552e-05,  6.89035498e-06, -6.05698680e-06, -2.48247011e-05,
        4.97528079e-05]), 9.519520714948009e-05, 5.845692330197494e-08]
Worst solution: [array([-0.03653993, -0.01903007, -0.01036523,  1.02774076,  0.12564404]), 0.5396705371870093, 1.0228380691204646]
Median solution: [array([ 0.01660673, -0.04630709, -0.0392979 , -0.11577761, -0.00823048]), 0.09995097996237945, 0.1887106187831269]
Mean solution: 0.35320463497058857
Standard deviation: 0.3249785661785129


### 7 variables

In [83]:
n = 7

In [84]:
m_EE1plus1_statistical_analysis(m, n, c, gens, k)

Best solution: [array([  3.00205151, -25.98795387,  19.02421249, -26.94559172,
        14.02777095, -21.01272896,   9.01286216]), 0.053981507926113115, 20.036261427642636]
Worst solution: [array([ -5.87915317, -16.88142218,  -3.15836586,  18.15460727,
        -9.98564469,   8.12323417,  27.25958527]), 0.9469433850679282, 20.891049645596834]
Median solution: [array([ 10.14654668,   8.05857595,  -3.91905205,  16.99966618,
       -12.05996534, -17.87963018,  -0.20205871]), 0.8160200925927548, 20.566038783928217]
Mean solution: 20.545965472204458
Standard deviation: 0.26825176912938126


In [85]:
m_EEmu_coma_lambda_statistical_analysis(m, n, gens, epsilon, mu, lambda_)

Best solution: [array([ 1.80002378e+01, -4.99579392e+00,  1.10004971e+01, -1.90016506e+01,
        3.36662956e-03, -1.30007513e+01,  2.59986802e+01]), 0.009172548774551656, 20.00026335169673]
Worst solution: [array([ 7.9361049 , 15.98289157, 21.0793486 ,  7.25479042, 14.1557304 ,
        7.02091564, 24.22745019]), 0.6469814444983447, 20.82889560326686]
Median solution: [array([-13.16163966,  27.09147456,  19.94806696,  20.11135184,
         2.06238446,  -0.04234687,   3.96345281]), 0.3054842129357746, 20.381490962804445]
Mean solution: 20.380822013695486
Standard deviation: 0.21792671545920445


In [86]:
m_EEmu_coma_lambda_2_statistical_analysis(m, n, gens, epsilon, mu, lambda_)

Best solution: [array([ 0.00058304,  0.00179647, -0.00039472,  0.00281797, -0.00141888,
        0.00113888, -0.00131012]), 0.004188448867268776, 0.00013747033154887234]
Worst solution: [array([ 0.75762464,  0.03517698,  0.04483801, -0.31599267,  0.01834183,
        0.03463856,  0.25856072]), 0.386975352957906, 1.491685339323205]
Median solution: [array([-0.121558  ,  0.01241594, -0.04518692, -0.14443212, -0.02328078,
        0.16164814, -0.24594171]), 0.22822170225314367, 0.7939395493196035]
Mean solution: 0.6207766256466491
Standard deviation: 0.5101122188512114


### 10 variables

In [87]:
n = 10

In [88]:
m_EE1plus1_statistical_analysis(m, n, c, gens, k)

Best solution: [array([ -6.01616428,   6.01665968, -16.06002686,  -1.05166803,
       -25.96696029,  27.97166021,  30.        , -12.98385506,
        13.08168364,  21.00085208]), 0.07839930598926104, 20.08150369746658]
Worst solution: [array([ -4.10934362,  29.08630942,  14.27614859,  18.93680351,
        -8.05485517,   4.54098702,   9.81133536,  29.09597835,
       -17.74283691, -30.        ]), 0.7282100406085759, 21.1473020157759]
Median solution: [array([-20.08536593, -21.00561993,  17.99607073,  20.30815499,
        18.99474869,  -9.06697992,  -0.28933524,   1.04813309,
       -17.89464385,  19.94188794]), 0.30997029326184433, 20.73523365837674]
Mean solution: 20.667933151262826
Standard deviation: 0.2906719403433458


In [89]:
m_EEmu_coma_lambda_statistical_analysis(m, n, gens, epsilon, mu, lambda_)

Best solution: [array([  4.03387927, -16.99557632,  15.99720601,  -9.00414947,
       -27.00497917, -25.00183903, -29.98266254,   5.99846789,
       -24.99389493,   4.04454297]), 0.02929349786597233, 20.018860629933503]
Worst solution: [array([ -1.20493687,  16.1317693 ,  27.89391421, -14.67212955,
        -8.78957998,  17.15183359,   2.95578513, -20.11416504,
       -14.09877964,   9.07085638]), 1.9527923395816387, 20.9802465744453]
Median solution: [array([-10.12149924,  -4.9768284 ,  -0.82423234, -26.87669789,
       -29.82664064,   5.04880466,  15.10686626,   5.09155564,
        19.13148476,  20.90574032]), 0.5883158908570983, 20.617714501965555]
Mean solution: 20.582735630067354
Standard deviation: 0.26750134607320547


In [90]:
m_EEmu_coma_lambda_2_statistical_analysis(m, n, gens, epsilon, mu, lambda_)

Best solution: [array([ 0.01498557, -0.0118652 ,  0.01347077,  0.00925267,  0.03307138,
        0.00797217,  0.01599471,  0.02837051, -0.02625752,  0.02346542]), 0.04191193505972319, 0.023447001441934123]
Worst solution: [array([ 0.07443409,  0.06332929,  0.1708825 ,  0.75692405, -0.6891745 ,
       -0.10924365, -0.78000691, -0.07482777, -0.09101098,  0.9518177 ]), 0.6965994297424397, 1.9894996468570594]
Median solution: [array([-0.17345257, -0.03691061, -0.11782934,  0.04490412,  0.16732914,
       -0.04219433,  0.06988423, -0.70935271,  0.0051102 , -0.4104745 ]), 0.41724129598291615, 1.3011397062038879]
Mean solution: 1.1865781735776766
Standard deviation: 0.480441637814099


m = 20     
### best Solution     

| Algorithm/Variables | $$2$$ |  $$5$$ | $$7$$ | $$10$$ |
| --- | --- | --- | --- | --- |
| $(1+1)-EE$ | 0.6011862010882258 | 20.001107809859562 | 20.036261427642636 | 20.08150369746658 |
| $(\mu, \lambda)- EE$ | 3.524187670267409e-05 | 0.01121256440277607 | 20.00026335169673 | 20.018860629933503 |
| $(\mu, \lambda)- EE*$ | 9.238938308175904e-06 | 5.845692330197494e-08 | 0.00013747033154887234 | 0.023447001441934123 |

### worst solution
| Algorithms/Variables | $$2$$ |  $$5$$ | $$7$$ | $$10$$ |
| --- | --- | --- | --- | --- |
| $(1+1)-EE$ | 20.09393931992774 | 20.712420824131915 | 20.891049645596834 | 21.1473020157759 |
| $(\mu, \lambda)- EE$ | 0.16740314303378812 | 20.53798145638035 | 20.82889560326686 | 20.9802465744453 |
| $(\mu, \lambda)- EE*$ | 0.04374946843978167 | 1.0228380691204646 | 1.491685339323205 | 1.9894996468570594 |   

### mean solution
| Algorithms/Variables | $$2$$ |  $$5$$ | $$7$$ | $$10$$ |
| --- | --- | --- | --- | --- |
| $(1+1)-EE$ | 19.051440173046764 | 20.384016904738335 | 20.545965472204458 | 20.667933151262826 |
| $(\mu, \lambda)- EE$ | 0.02358493990593393 | 6.472555886382096 | 20.380822013695486 | 20.582735630067354 |
| $(\mu, \lambda)- EE*$ | 0.009995132216457935 | 0.35320463497058857 | 0.6207766256466491 | 1.1865781735776766 |    

### std solution  
| Algorithms/Variables | $$2$$ |  $$5$$ | $$7$$ | $$10$$ |
| --- | --- | --- | --- | --- |
| $(1+1)-EE$ | 4.232848246692949 | 0.20615871338139483 | 0.26825176912938126 | 0.2906719403433458 |
| $(\mu, \lambda)- EE$ | 0.03927134480025898 | 8.982425850374126 | 0.21792671545920445 | 0.26750134607320547 |
| $(\mu, \lambda)- EE*$ | 0.0117439134574944 | 0.3249785661785129 | 0.5101122188512114 | 0.480441637814099 |